In [ ]:
import pymysql
import pymssql
import os
import pandas as pd
import datetime
import re
import numpy as np

In [ ]:
def ms_sql_con():
    sql_name = 'voice_ai'
    sql_server = '10.2.4.124'
    sql_login = 'ICECORP\\1c_sql'

    with open('sql.pass','r') as file:
        sql_pass = file.read().replace('\n', '')
        file.close()

    return pymssql.connect(
            server = sql_server,
            user = sql_login,
            password = sql_pass,
            database = sql_name,
        )
sql_conn = ms_sql_con()

In [ ]:
def calls_mysql_con_init():
    return pymysql.connect(
        host = '10.2.4.87',
        user = 'root',
        passwd = 'root',
        database = 'ml'
    )
calls_mysql_conn = calls_mysql_con_init()

In [ ]:
def master_mysql_con_init():
    with open('mysql.pass','r') as file:
        mysql_pass = file.read().replace('\n', '')
        file.close()
    return pymysql.connect(
        host = '10.2.4.146', 
        user = 'asterisk', 
        passwd = mysql_pass,
        db = 'MICO_96'
    )
master_mysql_conn = master_mysql_con_init()

In [ ]:
def call_mysql_con_init():
    with open('mysql.pass','r') as file:
        mysql_pass = file.read().replace('\n', '')
        file.close()
    return pymysql.connect(
        host = '10.2.4.146', 
        user = 'asterisk', 
        passwd = mysql_pass,
        db = 'asterisk'
    )
call_mysql_con = call_mysql_con_init()

In [ ]:
def get_file_list(path):
    file_list = []
    for root, dirs, files in os.walk(path):
        for file in files:
            file_list.append(os.path.join(root, file))
            #file_list.append(root+'/'+file)
    return file_list

def get_file_date(file_list):
    file_date = []
    for file in file_list:
        file_date.append(datetime.datetime.fromtimestamp(os.path.getmtime(file)))
    return file_date

def get_file_name(file_list):
    path_name = []
    file_name = []
    for file in file_list:
        #file_name.append(file)
        file_name.append(os.path.basename(file))        
        path_name.append(os.path.dirname(file))
    return path_name, file_name

In [ ]:
def get_files(path):
    file_list = get_file_list(path)
    file_date = get_file_date(file_list)
    path, file_name = get_file_name(file_list)
    df = pd.DataFrame({'path':path, 'file_name':file_name, 'file_date':file_date})
    df.sort_values(by='file_date', inplace=True)
    return df

#### Queue

In [ ]:
query = "SELECT column_name FROM information_schema.columns WHERE table_name='queue';"
df = pd.read_sql(query, sql_conn)
#df

In [ ]:
df = pd.read_sql('select date, filename from queue where source_id=1', sql_conn)
df.head(3)

#### Transcribations

In [ ]:
query = "SELECT column_name FROM information_schema.columns WHERE table_name='transcribations';"
df = pd.read_sql(query, sql_conn)
df

In [ ]:
query = 'select distinct linkedid, audio_file_name from transcribations '
query += "where audio_file_name='in_4999029955_2021-12-10-08-41-59rxtx.wav'"
df = pd.read_sql(query, sql_conn)
df

In [ ]:
query = 'select max(transcribation_date) from transcribations '
query += "where source_id=1"
df = pd.read_sql(query, sql_conn)
df

#### Calls

In [ ]:
query = "SELECT column_name FROM information_schema.columns WHERE table_name='calls';"
#df = pd.read_sql(query, calls_mysql_conn)

In [ ]:
### DELETE
#query = "delete from calls where call_date>'2021-12-10 00-00-00';"
#calls = pd.read_sql(query, calls_mysql_conn)

In [ ]:
query = "SELECT distinct"
query += " call_date, ak, miko, mrm, incoming, linkedid, base_name"
query += " FROM calls"
query += " where call_date>'2021-12-10 10-10-00'"
query += " order by call_date;"
calls = pd.read_sql(query, calls_mysql_conn)
#calls = pd.read_sql("SELECT distinct * FROM calls where linkedid='1639126809.114061' order by call_date;", calls_mysql_conn)
#calls = pd.read_sql("SELECT * FROM calls where linkedid='1639033806.82183' order by call_date;", calls_mysql_conn)
calls.head(3)

#### Cdr call

In [ ]:
query = "SELECT column_name FROM information_schema.columns WHERE table_name='cdr';"
df = pd.read_sql(query, call_mysql_con)
#for idx, row in df.iterrows():
#    print(row.column_name)

In [ ]:
"select calldate, src, dst from cdr where uniqueid = '" + uniqueid + "' limit 1;"

#### PT1C_cdr_MICO master

In [ ]:
query = "SELECT column_name FROM information_schema.columns WHERE table_name='PT1C_cdr_MICO';"
#query = "show tables;"
df = pd.read_sql(query, master_mysql_conn)
df

In [ ]:
query = "SELECT * from PT1C_cdr_MICO limit 1;"
df = pd.read_sql(query, master_mysql_conn)
df

#### Files call

In [ ]:
files_path = 'call/saved_for_analysis/debug/call'
dfc = get_files(files_path)
dfc.to_csv('files_call.csv', index=False)
dfc.head(3)

#### Files master

In [ ]:
files_path = 'call/saved_for_analysis/debug/master'
dfm = get_files(files_path)
dfm.to_csv('files_call.csv', index=False)
dfm.head(3)

### Relation: Calls - files

In [ ]:
calls.head(3)

In [ ]:
query = "select calldate, src, dst from cdr where uniqueid = '1639120182.110147' limit 1;"
df = pd.read_sql(query, call_mysql_con)
df

In [ ]:
query = "SELECT * from PT1C_cdr_MICO where uniqueid = '1639120182.110147' limit 1;"
dfcdr = pd.read_sql(query, master_mysql_conn)
dfcdr

In [ ]:
dfcdr.recordingfile.iloc()[0]

In [ ]:
dfc[dfc.file_name=='in_79269465675_2021-12-10-10-09-43.wav']#.iloc()[0]

In [ ]:
dfm[dfm.file_name=='in_79269465675_2021-12-10-10-09-43.wav']#.iloc()[0]

In [ ]:
query = 'select distinct linkedid, audio_file_name, record_date from transcribations '
query += "where record_date>'2021-12-10 10:10:00'"
#query += "where linkedid='1639120182.110147'"
#query += "where src='79269465675'"
trans = pd.read_sql(query, sql_conn)
trans.head(3)

In [ ]:
### Calls - transcribation relation
calls_trans = pd.merge(calls, trans, how='left', on='linkedid')
calls_trans['only_in_calls'] = calls_trans['audio_file_name'].isnull()
#calls_trans['only_in_dft'] = calls_trans['call_date'].isnull()
calls_trans.head(3)

In [ ]:
print(
    'С\n', 
    min(calls_trans[calls_trans.only_in_calls].call_date), 
    '\nПо\n', 
    max(calls_trans[calls_trans.only_in_calls].call_date), 
    '\n['+str(len(set(calls_trans[calls_trans.only_in_calls].linkedid)))+']',
    'Звонков без транскрибации'
)

In [ ]:
### Lost_calls - cdr_master
query = "SELECT * from PT1C_cdr_MICO where calldate >= '2021-12-10 10:10:02' and calldate <= '2021-12-10 12:27:26';"
dfcdr = pd.read_sql(query, master_mysql_conn)
dfcdr.head(3)

In [ ]:
lost_calls = calls_trans[calls_trans.only_in_calls]
cdr_master = dfcdr[{'linkedid','recordingfile'}]
lost_calls = pd.merge(lost_calls, cdr_master, how='left', on='linkedid')
lost_calls['not_found_in_cdr_master'] = lost_files_call['recordingfile'].isnull()
lost_calls.head(3)

In [ ]:
### Lost_calls cdr_call
query = "SELECT * from cdr where calldate >= '2021-12-10 10:10:02' and calldate <= '2021-12-10 12:27:26';"
dfcdr = pd.read_sql(query, call_mysql_con)
dfcdr.head(3)

In [ ]:
#lost_calls = calls_trans[calls_trans.only_in_calls]
cdr_call = dfcdr[{'uniqueid','recordingfile'}]
lost_calls = pd.merge(lost_calls, cdr_master, how='left', on='linkedid')
lost_calls['not_found_in_cdr_call'] = lost_files_call['recordingfile'].isnull()
lost_calls.head(3)

In [ ]:
def filename_normalize(name):
    if name == np.nan or name != name:
        return name    
    else:
        name = re.sub(r'^.*/', '', name)
        name = re.sub(r'\..*$', '', name)
        return name

In [ ]:
lost_calls.columns

In [ ]:
#lost_calls['recordingfile_x'] = lost_calls['recordingfile_x'].apply(filename_normalize)

In [ ]:
lost_calls.head(3)

In [ ]:
lost_linkedid = lost_calls[{'linkedid', 'recordingfile_x', 'recordingfile_y'}]

In [ ]:
#lost_linkedid.apply(pd.Series.nunique, axis=0)

In [ ]:
df = lost_linkedid.groupby(['linkedid', 'recordingfile_x', 'recordingfile_y']).max()
df.reset_index(inplace=True)
df.drop('recordingfile_y', axis=1, inplace=True)
df.head(3)
#for idx, row in df.iterrows():
#    print(row.linkedid, row.recordingfile_x)

In [ ]:
df.to_csv('файлы_не_попавшие_на_расшифровку.csv')

In [ ]:
# calldate >= '2021-12-10 10:10:02' and calldate <= '2021-12-10 12:27:26'

In [ ]:
crop_dfc = pd.DataFrame(dfc[(dfc.file_date>='2021-12-10 10:10:02') & (dfc.file_date<='2021-12-10 12:27:26')])
crop_dfc.drop('path', axis=1, inplace=True)
crop_dfc.to_csv('файлы_кц.csv')
crop_dfc.head(3)

In [ ]:
crop_dfm = pd.DataFrame(dfm[(dfm.file_date>='2021-12-10 10:10:02') & (dfm.file_date<='2021-12-10 12:27:26')])
crop_dfm.drop('path', axis=1, inplace=True)
crop_dfm.to_csv('файлы_мастеров.csv')
crop_dfm.head(3)